# Computing material intensity coefficients

This notebook deals with how to convert results from the model into material intensity coefficients for comparison with other studies.

## Setup

In [ ]:
import psycopg as pg
import pandas as pd

In [ ]:
params='dbname=macrocomponents3 user=postgres password=19Ni93co44PG!'

In [ ]:
material_types=['Metal','Concrete','Aggregates','Wood','Wool','Other','Glass','Cement_mortar','Gypsum_plaster','Clay']
elements=['ext_wall','int_wall','floor','foundation','roof_cover','roof_structure','window','ridge_board', 'top_floor_ceiling', 'ground_slab']

In [ ]:
def run_sql (DBparameters,SQLcode):
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(DBparameters)

        # create a new cursor
        cur = connector.cursor()

        # execute the SQL statement
        cur.execute(SQLcode)

        # commit the changes to the database
        connector.commit()

        # close communication with the database
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)

    finally:
        if connector is not None:
            connector.close()

## Block-wise Material intensity coefficients

This section computes block-wise material intensity coefficients in the same format as Lanau & Liu (2020) "Developing an Urban Resource Cadaster for Circular Economy: A Case of Odense, Denmark". Material intensity coefficients are calculated for the "under-block", corresponding to all underground elements, the "vertical block" corresponding to walls, the "horizontal block" corresponding to intermediate floors, and the "over-block" corresponding to the roof, attic, etc. The OB and UB MICs are calculated per m2 footprint, whereas the VB MIC is per m2 footprint and per floor, and the HB MIC per m2 of intermediate floors.

In [ ]:
def materials_per_element(material_types, elements):
    SQL="""CREATE TABLE IF NOT EXISTS materials_per_element (bbr_id varchar(50) PRIMARY KEY, """
    
    for m in material_types:
        for e in elements:
            SQL+=m+'_'+e+' real, '
    SQL=SQL[0:len(SQL)-2]+'); '
    
    SQL+="""WITH t AS(
    SELECT
    rma.bbr_id,
    rma.element,
    rma.product,
    weight,
    material_type
    FROM results_material_amounts rma
    INNER JOIN products pr ON rma.product=pr.name,
    LATERAL (SELECT (CASE WHEN unit='KG' THEN amount WHEN unit='M3' THEN amount*pr.density ELSE NULL END) AS weight) lt
    )

    SELECT
    bbr_id,
    SUM(weight) weightsum,
    material_type,
    element
    INTO TEMP TABLE t2
    FROM t
    GROUP BY material_type, bbr_id, element
    ORDER BY bbr_id;
    
    DELETE FROM materials_per_element;

    """
    for m in material_types:
        for e in elements:
            SQL+="INSERT INTO materials_per_element(bbr_id, "+m+'_'+e+") SELECT bbr_id, weightsum FROM t2 WHERE material_type = '"+m+"' AND element = '"+e+"' ON CONFLICT ON CONSTRAINT materials_per_element_pkey DO UPDATE SET (bbr_id, "+m+'_'+e+") = (EXCLUDED.bbr_id, EXCLUDED."+m+'_'+e+"); "

    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute(SQL)

        conn.commit()
        cur.close()
        
    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()     

In [ ]:
materials_per_element(material_types, elements)

In [ ]:
def mics(material):
    dic={'ob':[],'hb':[],'ub':[],'vb':[]}
    index=[]
    
    SQL="""SELECT 
    b.id_lokalid,
    ob,
    hb,
    ub,
    vb
    FROM materials_per_element mpe
    INNER JOIN buildings b ON b.id_lokalid=mpe.bbr_id,
    LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0 AND b.byg054AntalEtager>0) THEN (COALESCE(%s_ext_wall,0)+COALESCE(%s_int_wall,0)+COALESCE(%s_window,0))/(b.byg041BebyggetAreal*b.byg054AntalEtager) ELSE NULL END) AS vb) lvb,
    LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0) THEN (COALESCE(%s_roof_structure,0)+COALESCE(%s_roof_cover,0)+COALESCE(%s_ridge_board,0)+COALESCE(%s_top_floor_ceiling,0)+COALESCE(%s_top_floor_ceiling,0))/b.byg041BebyggetAreal ELSE NULL END) AS ob) lob,
    LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0) THEN (COALESCE(%s_foundation,0)+COALESCE(%s_ground_slab,0))/b.byg041BebyggetAreal ELSE NULL END) AS ub) lub,
    LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0 AND b.byg054AntalEtager>1) THEN COALESCE(%s_floor,0)/(b.byg041BebyggetAreal*(b.byg054AntalEtager-1)) ELSE NULL END) AS hb) lhb
    ORDER BY id_lokalid ASC""" % (material,material,material,material,material,material,material,material,material,material,material)
    
   
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute(SQL)
        
        row=cur.fetchone()     
              
        while row is not None:
            index.append(row[0])
            n=1
            for k in dic.keys():
                dic[k].append(row[n])
                n+=1

            row=cur.fetchone()
            
        results=pd.DataFrame(dic,index)

        conn.commit()
        cur.close()
        
        return(results)
        
    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()   

In [ ]:
m=mics('clay')

In [ ]:
m

In [ ]:
def all_mics(materials):
    dic={}
    index=[]

    for m in materials:
        ob=m+'_ob'
        hb=m+'_hb'
        ub=m+'_ub'    
        vb=m+'_vb'
    
        dic[ob]=[]
        dic[hb]=[]
        dic[ub]=[]
        dic[vb]=[]
    
        SQL="""SELECT 
        b.id_lokalid,
        ob,
        hb,
        ub,
        vb
        FROM materials_per_element mpe
        INNER JOIN buildings b ON b.id_lokalid=mpe.bbr_id,
        LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0 AND b.byg054AntalEtager>0) THEN (COALESCE(%s_ext_wall,0)+COALESCE(%s_int_wall,0)+COALESCE(%s_window,0))/(b.byg041BebyggetAreal*b.byg054AntalEtager) ELSE NULL END) AS vb) lvb,
        LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0) THEN (COALESCE(%s_roof_structure,0)+COALESCE(%s_roof_cover,0)+COALESCE(%s_ridge_board,0)+COALESCE(%s_top_floor_ceiling,0)+COALESCE(%s_top_floor_ceiling,0))/b.byg041BebyggetAreal ELSE NULL END) AS ob) lob,
        LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0) THEN (COALESCE(%s_foundation,0)+COALESCE(%s_ground_slab,0))/b.byg041BebyggetAreal ELSE NULL END) AS ub) lub,
        LATERAL (SELECT (CASE WHEN (b.byg041BebyggetAreal>0 AND b.byg054AntalEtager>1) THEN COALESCE(%s_floor,0)/(b.byg041BebyggetAreal*(b.byg054AntalEtager-1)) ELSE NULL END) AS hb) lhb
        ORDER BY id_lokalid ASC""" % (m,m,m,m,m,m,m,m,m,m,m)


        try:
            conn = pg.connect(params, row_factory=pg.rows.dict_row)
            cur=conn.cursor()

            cur.execute(SQL)

            row=cur.fetchone()     

            while row is not None:
                if m==materials[0]:
                    index.append(row['id_lokalid'])

                dic[ob].append(row['ob'])
                dic[hb].append(row['hb'])
                dic[ub].append(row['ub'])
                dic[vb].append(row['vb'])

                row=cur.fetchone()

            cur.close()

        except (Exception, pg.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()           
            
    results=pd.DataFrame(dic,index)
    return(results)

In [ ]:
All_MICs=all_mics(material_types)

#### Reading data from SDU model:

In [ ]:
SDU=pd.read_csv('C:/Users/KJ35FA/Downloads/bhuvans data.csv', sep=';')

In [ ]:
def lowercase(str):
    return str.lower()

In [ ]:
SDU['bygning_id']=SDU['bygning_id'].transform(lowercase)

In [ ]:
SDU.index=[x.lower() for x in SDU['bygning_id']]

#### Computing MICs per archetype

In [ ]:
AMSDU=pd.merge(All_MICs, SDU, left_index=True, right_index=True, how='inner')


In [ ]:
AMSDU.describe()

In [ ]:
archetypes=list(dict.fromkeys(AMSDU['archetype']))
archetypes

In [ ]:
years=[0, 1850, 1930, 1950, 1960, 1972, 1978, 1998, 2006, 2010, 2100]

In [ ]:
mic_dic={}

for a in archetypes:
    for n in range(1,len(years)):
        ymax=str(years[n])
        ymin=str(years[n-1])
        name=a+ymax
        q='(archetype == "'+a+'") & (opfoerelse_aar <= '+ymax+') & (opfoerelse_aar > '+ymin+')'
        mic_dic[name]=AMSDU.query(q)

In [ ]:
mic_dic['SFH1850']

In [ ]:
mic_means={}
for k in mic_dic.keys():
    means=mic_dic[k].mean(numeric_only=True)
    mic_means[k]=means

In [ ]:
mic_meds={}
for k in mic_dic.keys():
    meds=mic_dic[k].median(numeric_only=True)
    mic_meds[k]=meds

In [ ]:
mic_10={}
for k in mic_dic.keys():
    q=mic_dic[k].quantile(0.1)
    mic_10[k]=q

In [ ]:
mic_90={}
for k in mic_dic.keys():
    q=mic_dic[k].quantile(0.9)
    mic_90[k]=q

In [ ]:
mic_std={}
for k in mic_dic.keys():
    std=mic_dic[k].std(numeric_only=True)
    mic_std[k]=std

In [ ]:
MICS={}
for k in mic_dic.keys():
    d={}
    d['mean']=mic_means[k]
    d['median']=mic_meds[k]
    d['10q']=mic_10[k]
    d['90q']=mic_90[k]
    d['std']=mic_std[k]
    MICS[k]=d

In [ ]:
MICS['SFH1850']['mean']

In [ ]:
mics_list=[]
for k in MICS.keys():
    df=pd.DataFrame({k:MICS[k]['mean']})
    mics_list.append(df)
mics_table=pd.concat(mics_list, axis=1)

In [ ]:
mics_table.to_excel('C:/Users/KJ35FA/Documents/Python/MICS0603.xlsx')

# Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
boxplot=AMSDU.boxplot(column=['Clay_vb','Concrete_vb','Wood_vb'])

In [ ]:
boxplot.figure

In [ ]:
import seaborn as sns

In [ ]:
kde=sns.displot(AMSDU,x='Concrete_vb')

In [ ]:
kde.set(ylim=(0,350))

In [ ]:
kde.figure

## Material intensity coefficients per m2

This section computes MICs at the building level in kg material per m2 of floor area. This is a common indicator, useful for comparison with previous studies.

In [ ]:
def all_mics2(materials):
    dic={}
    for m in materials:
        dic[m]=[]
    index=[]
    SQL="SELECT mpe.*, b.byg038SamletBygningsareal FROM materials_per_element mpe INNER JOIN buildings b ON b.id_lokalid=mpe.bbr_id ORDER BY id_lokalid ASC"
    conn=None 
    
    try:
        conn = pg.connect(params, row_factory=pg.rows.dict_row)
        cur=conn.cursor()

        cur.execute(SQL)

        row=cur.fetchone()
        
        while row is not None:

            for m in materials:
                material_amount=0    
                if m==materials[0]:
                    index.append(row['bbr_id'])
                    
                for k in row.keys():
                    if m in k:
                    material_amount+=row[k]    

                dic[m].append(material_amount/row['byg038SamletBygningsareal'])

            row=cur.fetchone()

            cur.close()

        except (Exception, pg.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()           
            
    results=pd.DataFrame(dic,index)
    return(results)